# 05 - Time-to-Failure (TTF) Model Training

This notebook trains a neural network regression model to predict how many hours remain before a battery failure. This predictive maintenance capability helps anticipate battery issues before they occur.


## Disable GPU Usage

Configure TensorFlow to use CPU only for consistent execution.


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## Import Required Libraries

Import all necessary libraries for data processing, model building, and evaluation.


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
import joblib
import subprocess
import shutil
import json

2025-11-20 20:57:33.035411: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/opt/app-root/lib64/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.

## Load Data and Calculate Time Before Failure

Load the battery data and calculate the time remaining until failure for each timestamp by measuring the difference from the maximum timestamp in the dataset.


In [4]:
df = pd.read_csv("./data/battery_data.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["timeBeforeFailure"] = (df["timestamp"].max() - df["timestamp"]).dt.total_seconds() / 3600

## Select Features and Target Variable

Choose the most relevant battery parameters as input features and the calculated time-before-failure as the target for regression.


In [5]:
features = ["batteryTemp", "batteryCurrent", "batteryVoltage", "stateOfCharge", "stateOfHealth"]
target = "timeBeforeFailure"

X = df[features]
y = df[target]

## Split and Normalize Data

Split the data into training and testing sets, then normalize the features to have zero mean and unit variance.


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Build Regression Neural Network

Create a neural network with three hidden layers for predicting continuous time-to-failure values.


In [7]:
model = keras.Sequential([
    keras.layers.Dense(128, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dense(1) 
])

model.compile(optimizer="adam", loss="mse", metrics=["mae"])

/opt/app-root/lib64/python3.12/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-11-20 20:58:18.466657: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


## Train the Model

Train the regression model for 50 epochs, using the test data for validation during training.


In [8]:
model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test))

Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.1532 - mae: 0.3066 - val_loss: 0.0430 - val_mae: 0.1688
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0336 - mae: 0.1442 - val_loss: 0.0157 - val_mae: 0.1029
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0144 - mae: 0.0978 - val_loss: 0.0094 - val_mae: 0.0776
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0077 - mae: 0.0716 - val_loss: 0.0048 - val_mae: 0.0570
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0042 - mae: 0.0522 - val_loss: 0.0031 - val_mae: 0.0446
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0033 - mae: 0.0455 - val_loss: 0.0023 - val_mae: 0.0378
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0027 - mae: 0.0413 - val_loss: 0.0022 - val_mae: 0.0375
Epoch 8/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0023 - mae: 0.0380 - val_loss: 0.0016 - val_mae: 0.0312
Epoch 9/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - mae: 

## Make Predictions on Test Data

Use the trained model to predict time-to-failure on the test set.


In [9]:
y_pred = model.predict(X_test_scaled, verbose=0).flatten()

## Save Model and Scaler

Save the trained TTF model and scaler for later use in predictions.


In [11]:
os.makedirs("models", exist_ok=True)
scaler_path = "models/ttf_scaler.pkl"
joblib.dump(scaler, scaler_path)
model.save("models/battery_ttf_model.keras")

## Export Model to TensorFlow SavedModel Format

Export the model in TensorFlow's SavedModel format for conversion to OpenVINO IR.


In [12]:
try:
    model.export("models/battery_ttf_model")
except AttributeError:
    model.save("models/battery_ttf_model", save_format='tf')

INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


INFO:tensorflow:Assets written to: models/battery_ttf_model/assets


Saved artifact at 'models/battery_ttf_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 5), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140288182949584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182950736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182948048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182948432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182951312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182949008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182951696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140288182950544: TensorSpec(shape=(), dtype=tf.resource, name=None)


## Convert Model to OpenVINO IR Format

Convert the TensorFlow model to OpenVINO IR for optimized inference on Intel hardware.


In [13]:
os.makedirs("models/battery_ttf_model_ir", exist_ok=True)

subprocess.run(
    [
        "ovc",
        "models/battery_ttf_model",
        "--output_model", "models/battery_ttf_model_ir/saved_model"
    ],
    capture_output=True,
    text=True,
    check=True
)

CompletedProcess(args=['ovc', 'models/battery_ttf_model', '--output_model', 'models/battery_ttf_model_ir/saved_model'], returncode=0, stdout='[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression by removing argument "compress_to_fp16" or set it to false "compress_to_fp16=False".\nFind more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html\n[ SUCCESS ] XML file: models/battery_ttf_model_ir/saved_model.xml\n[ SUCCESS ] BIN file: models/battery_ttf_model_ir/saved_model.bin\n', stderr='')

## Prepare Model for Serving

Organize the IR model files into a versioned directory structure for OpenVINO Model Server.


In [14]:

def prepare_ir_model_for_serving(ir_model_path, output_path, version=1):
    version_path = os.path.join(output_path, str(version))
    os.makedirs(version_path, exist_ok=True)
    
    for item in os.listdir(ir_model_path):
        if item.endswith(('.xml', '.bin')):
            src = os.path.join(ir_model_path, item)
            dst = os.path.join(version_path, item)
            shutil.copy2(src, dst)
    
    return version_path

serving_path = "models/serving/battery_ttf_model"
ir_model_path = "models/battery_ttf_model_ir"

model_version_path = prepare_ir_model_for_serving(ir_model_path, serving_path, version=1)
